In [74]:
import numpy as np
import random

import torch
import torch.utils
from torch import optim
from torch import nn
from torch.nn import functional as F

import gobanana as gb

In [97]:
class NaiveGenerator(gb.nn.Generator):
    def __init__(self, board_shape, num_metrics, hidden_dim):
        super().__init__(board_shape, num_metrics)
        board_size = board_shape[0] * board_shape[1]
        input_size = board_size + num_metrics
        self.fc1 = nn.Linear(input_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, board_size * 3)

    def forward(self, noise: torch.Tensor, metrics: torch.Tensor):
        x = torch.cat([noise.flatten(start_dim=1), metrics.flatten(start_dim=1)])
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = x.reshape(*self.board_shape, 3)
        x = F.softmax(x, dim=-1)
        return x

In [98]:
class NaiveEvaluator(gb.nn.Evaluator):
    def __init__(self, board_shape, num_metrics, hidden_dim):
        super().__init__(board_shape, num_metrics)
        board_size = board_shape[0] * board_shape[1]
        self.embed = nn.Embedding(3, 1)
        self.fc1 = nn.Linear(board_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, num_metrics)
        
    def forward(self, one_hot_boards):
        x = one_hot_boards
        print(self.embed(x).shape)
        x = self.embed(x).flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [99]:
def batch_generate_boards(generator, board_shape, metrics, batch_size=32):
    generator.eval()
    with torch.no_grad():
        noise = torch.rand(batch_size, *board_shape)
        metrics = torch.tensor(metrics).float()
        generator_out = generator(noise, metrics)
        board_mat = generator_out.argmax(-1).numpy()
        return gb.game.Board(board_mat)
    
def count_bananas(board: gb.game.Board):
    return np.sum(board.mat == board.BANANA).item()

def get_training_samples(boards):
    training_samples = []
    for board in boards:
        actual_num_bananas = count_bananas(board)
        board_one_hot_tensor = torch.tensor(board.mat).long()
        metrics_tensor = torch.tensor(actual_num_bananas).float()
        sample = (board_one_hot_tensor, metrics_tensor)
        training_samples.append(sample)
    return training_samples

In [100]:
def train_evaluator(evaluator, training_samples, epochs=1):
    optimizer = optim.Adam(evaluator.parameters())
    loss_function = nn.MSELoss()
    evaluator.train()
    for _ in range(epochs):
        for board_one_hot_tensor_, accurate_metrics in training_samples:
            optimizer.zero_grad()
            predicted_metrics = evaluator(board_one_hot_tensor_)
            loss = loss_function(accurate_metrics, predicted_metrics)
            loss.backward()
            optimizer.step()
            print('loss:', loss.item())

In [101]:
board_shape = (3, 3)
generator = NaiveGenerator(board_shape, 1, 32)
evaluator = NaiveEvaluator(board_shape, 1, 32)

num_boards_to_generate = 1024
generated_boards = []
for _ in range(num_boards_to_generate):
    desired_num_bananas = random.randint(0, 9)
    metrics = [desired_num_bananas]
    generated_board = batch_generate_boards(generator, board_shape, metrics)
    generated_boards.append(generated_board)
    
batch_size = 32
    
training_samples = get_training_samples(generated_boards)
train_loader = torch.utils.data.DataLoader(
        training_samples,
        batch_size=batch_size
)
train_evaluator(evaluator, train_loader, epochs=10)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)